<a href="https://colab.research.google.com/github/megredgar/Beta-div/blob/master/test_embHEtools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import sys
import os
import sqlite3
import pandas as pd
import embHEtools

In [5]:
os.chdir("C:/Users/megre/HawkEars")

In [6]:
projectname="MAWAprj"

In [7]:
spp_to_include=["MAWA"]

In [8]:
orgid=fr"ABMI/ARU"
projid=fr"ABMI-EH"
yearid=2022
siteid=fr"228"
runid=fr"{projectname}_{projid}_{siteid}_{yearid}"
audio_dir=fr"//nfs3.wildtrax.ca/BUpublic/{orgid}/{projid}/{yearid}/V1/{siteid}"

In [9]:
min_mmdd=501
max_mmdd=504
min_time=50000
max_time=70000

In [14]:
cutoff=0.01
overlap=0
merge=0

In [15]:
python = fr"C:/Users/megre/HawkEars/hawkears_venv/Scripts/python.exe"

In [16]:
hawkears_code = fr"C:/Users/megre/HawkEars/analyze.py"

In [18]:
he_allspp_cv = "C:/Users/megre/HawkEars/species_codes_morethanbirds.csv"

In [26]:
# Locations of input files required by HawkEars
output_dir = fr"G:/My Drive/test_hawkears/{runid}"
#After crawling server this is the folder where text files from each run are going to be stored

fileall = fr"{output_dir}/{runid}_files.csv" # Location of all files found in the selected audio_dir
filecount = fr"{output_dir}/{runid}_countfiles.csv" # Location of counts of wac/wav files per location
fileformat = fr"{output_dir}/{runid}_formatfiles.csv" # Location of all files, formatted for selection
filesubset = fr"{output_dir}/{runid}_subsetfiles.csv"
# Location of selected recordings based on date/time

filemerge = fr"{output_dir}/{runid}_mergefiles.csv" # Location of merged count and subset data
filelistoflists = fr"{output_dir}/{runid}_listoflists.csv"
#Location of the list of files you are going to send to HawkEars the directory they are in



In [27]:
# Locations of output files from HawkEars
tag_dir = fr"G:/My Drive/test_hawkears/tags/{runid}"

# Tags created by HawkEars in raw CSV
foldertags = fr"{tag_dir}/{runid}" # Location of where HawkEars writes tags in CSV format

database_name = fr"G:/My Drive/test_hawkears/hawkears_{projectname}_database.db"
# Where results are stored in addition to raw CSV

rootdir_subset = fr"G:/My Drive/test_hawkears/hawkears_outputs"
# Loops through this folder structure to find every file processed you selected for HawkEars to run and writes the path to database

# Location of logs
os.makedirs(foldertags, exist_ok=True)
log_file_path = fr"{foldertags}/{runid}_output_log.txt"  # Log file location
log_file = open(log_file_path, "w", buffering=1, encoding="utf-8")  # Line-buffered

In [34]:
# Workflow
# Send logs to screen and text file
from embHEtools import Tee
Tee = Tee(log_file_path)
sys.stdout = Tee
sys.stderr = Tee

# Settings used for this run that are tracked in the log
print(fr"Focal species are: {spp_to_include}")
print(fr"Minimum date of year is: {min_mmdd}")
print(fr"Maximum date of year is: {max_mmdd}")
print(fr"Minimum time of day is: {min_time}")
print(fr"Maximum time of day is: {max_time}")
print(fr"HE score cutoff value is: {cutoff}")
print(fr"Overlap value is: {overlap}")
print(fr"Merge value is: {merge}")


ImportError: cannot import name 'Tee' from 'embHEtools' (unknown location)

In [32]:
# 0. Sets species you want HawkEars to scan for in recordings
embHEtools.hawkears_selectspp(he_allspp_cv, spp_to_include)

# 1. Scans all folders on your storage space recursively and writes paths to wav and wac files. Only wav processed currently. Stored as CSV.
embHEtools.audiolist_create(audio_dir, fileall)

# 2. Create a list of all locations and count # of wac vs wav files. Only wav processed currently. Stored as CSV.
embHEtools.audiolist_count(fileall, filecount)

# 3. Formats the full list of audio files to allow selection of recordings with certain properties (retains path). Stored as CSV
embHEtools.audiolist_format(fileall, fileformat)

# 4. Select the mmdd (aka recording_date without year) and recording_time(s) you want. This subset stored as CSV
embHEtools.audiolist_filter(fileformat, min_mmdd, max_mmdd, min_time, max_time, filesubset)

# 5. Joins count data to subset data and saves to filemerge. Stored as CSV
embHEtools.audiolist_join(filecount, filesubset, filemerge)

# 6. Create individual HawkEars lists to run for each location. Stored as CSV
embHEtools.audiolist_filelist(filemerge, input_dir)

# 7. Make a master list of filelists. Filelists are the recordings that you select for Hawk Ears to process. Stored as CSV
embHEtools.audiolist_listoflists(filemerge, filelistoflists, input_dir, foldertags)




NameError: name 'embHEtools' is not defined

In [ ]:
# 8. Run HawkEars and populates a SQL database
embHEtools.hawkears_run(database_name, filelistoflists, python, hawkears_code, cutoff, overlap, merge)

# 9. Create/update SQL database table that has all required fields for fields created by Hawk Ears. Called hawkears_results in your SQL database
embHEtools.hawkears_dbaseupdate(database_name)

#10. Create/ update database table that has a list of all files that you attempted to run and the path to those files. Called all_subsetfiles in your SQLdatabase
embHEtools.hawkears_filesrun(rootdir_subset, database_name)

# Shut down log system
sys.stdout.flush()
sys.stderr.flush()
Tee.close()

# Completed automated recognition workflow
print("✅ Processing complete! Log file saved to:", log_file_path)



In [ ]:
#@title Create vtbl(s)
# Validation tables are created if they do not exist. If they exist, they are updated with data that is not duplicates.
# If duplicates (i.e you reran same files using same settings the labels are not written to the database because they exist)
import embHEtools
import sqlite3
import pandas as pd

# Database and table names
database_name = "G:/Shared drives/Bayne_Research/projects/oven_arrival/hawkears_tewa999_database.db" # Name of the SQLite database to call hawkears_results from

# Validation settings
top_n = 1 # Number of best tags to keep in a recording (filename) for validation. All tags above cutoff are kept in hawkears_results table, this is just for validation. Default validation setting is all labels
min_spacing = 30 # Approximate number of seconds between the tags selected for validation. Goal here is to make sure validated songs are unique and not HawkEars scoring same song as the window slides.
validation_type = 2  # Choose from: 1=bestlocationlabel, 2=firstlocationlabel, 3=bestfilelabel, 4=firstfilelabel, 5=minmaxfilelabel, 6=gradientfilelabel, 7=alllabels, 8=firstlocationlabel_alltop_n
validation_table = (fr"vtbl{validation_type}") # Automatic naming
labelsperbin = 10 # For gradientfilelabel method (# 6)

# Create validation dataset
embHEtools.validate_maketables(database_name=database_name, validation_table=validation_table, top_n = top_n, min_spacing = min_spacing, validation_type=validation_type, labelsperbin=3)



In [ ]:
#@title Validate data
import embHEtools

# Database and table names
database_name = "G:/Shared drives/Bayne_Research/projects/oven_arrival/hawkears_TEWA999_database.db" # Name of the SQLite database to call hawkears_results from
vtype = 2  # Choose from: 1=bestlocationlabel, 2=firstlocationlabel, 3=bestfilelabel, 4=firstfilelabel, 5=minmaxfilelabel, 6=gradientfilelabel, 7=alllabels
class_code = 'OVEN'
validation_table = (fr"vtbl{vtype}_{class_code}") # Automatic naming

embHEtools.validator(
    database_name=database_name,
    validation_table=validation_table,
    class_code=class_code,
    buffer=1,
    min_freq=3000,
    max_freq=10000,
    n_fft=512,
    hop_length=256,
    target_sr=22000,
    use_setduration=False,
    set_duration=10.0
)

